In [1]:
pip install gradio

     ---------------------------------------- 19.8/19.8 MB 4.3 MB/s eta 0:00:00
     ------------------------------------ 124.7/124.7 kB 665.3 kB/s eta 0:00:00
     -------------------------------------- 471.5/471.5 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 289.0/289.0 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 59.6/59.6 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 87.5/87.5 kB 990.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 45.7/45.7 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 65.7/65.7 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 319.8/319.8 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 75.4/75.4 kB 4.1 MB/s eta 0:00:00
     -------------------------------------- 364.0/364.0 kB 5.6 MB/s eta 0:00:00
     -------------

In [3]:
import gradio as gr

In [2]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

In [4]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.0.attn.masked_bias', 'decoder.transformer.h.2.attn.masked_bias', 'decoder.transformer.h.3.attn.bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.1.crossattention.masked_bias', 'decoder.transformer.h.6.crossattention.bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.4.crossattention.masked_bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.6.attn.masked_bias', 'decoder.transformer.h.8.attn.bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.6.attn.bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.2.crossattention.bias', 'decoder

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [32]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

In [33]:
def predict_step(image):
    pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    
    output_ids = model.generate(pixel_values, **gen_kwargs)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

In [34]:
demo = gr.Interface(fn=predict_step, 
             inputs=gr.inputs.Image(),
             outputs=gr.outputs.Textbox(label="Generated Text"))

C:\Users\Mohammad Junaid Alam\AppData\Local\Temp\ipykernel_5096\3867597136.py:2: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(),
C:\Users\Mohammad Junaid Alam\AppData\Local\Temp\ipykernel_5096\3867597136.py:2: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(),
C:\Users\Mohammad Junaid Alam\AppData\Local\Temp\ipykernel_5096\3867597136.py:3: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(label="Generated Text"))


In [35]:
demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
